In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import torch 
import os
import json
import math

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tools.models import simpleLSTM, simpleLSTM_quantiles
from tools.utils import run_closed_loop, perf_measure, run_closed_loop_quantile

import matplotlib.pyplot as plt


### Initialize and load already trained model

In [8]:
settlement      = "Rural-LV1-101-2034"
experiment      = "BaseScenario_FCInterpolLin" #BaseScenari
time_horizon    = "0101-3112"
target_model    = "EC_NP_LV_real_irradiance_MSE"
target          = "LV"
model_name      = f'{target_model}_best_valid_model.pt'
n_outputs       = 3
use_positional_encoding = 'all'
loss_type = 'mse' # 'mse' or 'quantile'

if n_outputs > 1:
    multi_output = True
else:
    multi_output = False

experiment_path = os.path.join('./experiments', settlement, experiment, time_horizon)
lstm_data_path  = os.path.join(experiment_path,"analysis", "lstm", "data")
lstm_model_path = os.path.join(experiment_path, "analysis", "lstm", "model")

model_file = os.path.join(lstm_model_path, model_name)

if loss_type == 'quantile':
    model = simpleLSTM_quantiles(n_input_features=4, n_hidden=50, num_layers=4, n_outputs=n_outputs)
    model.load_state_dict(torch.load(model_file,map_location=torch.device("cpu")))
    run_inference = run_closed_loop_quantile
elif loss_type == 'mse':
    model = simpleLSTM(n_input_features=4, n_hidden=50, num_layers=4, n_outputs=n_outputs)
    model.load_state_dict(torch.load(model_file,map_location=torch.device("cpu")))
    run_inference = run_closed_loop
else:
    raise NotImplementedError("Specify correct loss type")



#### Load scaled data and scalings

In [9]:
# Read test data
df_test_scaled_list_LV = []
for i in range(6):
    test_data_path = os.path.join(lstm_data_path, f'{target}_scaled_by_LV_test{i}.csv')
    df_test_scaled = pd.read_csv(test_data_path, index_col = 0)
    # Convert timestamps to datetime objects 
    datetime_index = pd.to_datetime(df_test_scaled.index)
    df_test_scaled.index = datetime_index
    df_test_scaled_list_LV.append(df_test_scaled)

df_test_scaled_list_NP = []
for i in range(6):
    test_data_path = os.path.join(lstm_data_path, f'NP_scaled_by_LV_test{i}.csv')
    df_test_scaled = pd.read_csv(test_data_path, index_col = 0)
    # Convert timestamps to datetime objects 
    datetime_index = pd.to_datetime(df_test_scaled.index)
    df_test_scaled.index = datetime_index
    df_test_scaled_list_NP.append(df_test_scaled)

df_test_scaled_list_EC = []
for i in range(6):
    test_data_path = os.path.join(lstm_data_path, f'EC_scaled_by_LV_test{i}.csv')
    df_test_scaled = pd.read_csv(test_data_path, index_col = 0)
    # Convert timestamps to datetime objects 
    datetime_index = pd.to_datetime(df_test_scaled.index)
    df_test_scaled.index = datetime_index
    df_test_scaled_list_EC.append(df_test_scaled)

# Read scalings
scaling_data_path = os.path.join(lstm_data_path,f'{target}_scalings_by_LV.json')
with open(scaling_data_path, 'r') as file:
    scaling_dict = json.load(file)

## Experiments we want to analyze
We extend our network to predict more finegrained information. On a Non-participants and Energy-Community participants level. Additionally we incorporate a Qunatille Loss based Uncertainty estimation for the predictions
- Forecasting performance MSE vs Qunatile Loss(Median) - on a NP, EC, LV Level 
- Training with real irradiance or with forecasted irradiance - compare results based on metrics (MAE, RMSE)
- Evaluate the quality of the uncertainty bounds (percentage in interval)


### Midterm-forecasting comparison 
Between irradiance forecasts and (imaginary result of having the original irradiance)

In [10]:
# prepare scalings to rescale original power profile:
max_power = scaling_dict[target]['max']
min_power = scaling_dict[target]['min']
scalings_max_min = (max_power, min_power)

months = ['july', 'august', 'september', 'october', 'november', 'december']
source_order = ['NP', 'EC', 'LV']


In [11]:
if loss_type == 'quantile':
    df_metrics_comparison = []
    show_figures = False
    for i in range(6):
        lookback = 96
        test_values_LV = df_test_scaled_list_LV[i].values
        groundtruth_LV = test_values_LV[lookback+1::, 0] # There is a + 1 shift in the inference loop
        groundtruth_NP = df_test_scaled_list_NP[i].values[lookback+1::, 0]
        groundtruth_EC = df_test_scaled_list_EC[i].values[lookback+1::, 0]

        num_test_samples = test_values_LV.shape[0]
        steps2predict = num_test_samples-lookback
        
        #groundtruth = test_values[lookback:-1, 0] 
        time = pd.Series(df_test_scaled_list_LV[i].index)[lookback:-1]
        predictions_with_irradiance_fc, predictions_with_irradiance_fc_quantiles = run_inference(model, test_values_LV, lookback=lookback, 
                                                                                                            future_prediction=steps2predict, 
                                                                                                            use_positional_encoding='all', 
                                                                                                            multi_output=multi_output, 
                                                                                                            )

        predictions_with_irradiance_original, predictions_with_irradiance_original_quantiles = run_inference(model, test_values_LV, lookback=lookback, 
                                                                                                                        future_prediction=steps2predict,    
                                                                                                                        use_positional_encoding='all_original', 
                                                                                                                        multi_output=multi_output,
                                                                                                                        )

        groundtruth_LV = groundtruth_LV * (max_power - min_power) + min_power #Rescale to orignal values
        groundtruth_NP = groundtruth_NP * (max_power - min_power) + min_power #Rescale to orignal values
        groundtruth_EC = groundtruth_EC * (max_power - min_power) + min_power #Rescale to orignal values

        predictions_with_irradiance_fc = predictions_with_irradiance_fc * (max_power - min_power) + min_power #Rescale to orignal values
        predictions_with_irradiance_original = predictions_with_irradiance_original * (max_power - min_power) + min_power #Rescale to orignal values

        predictions_with_irradiance_fc_quantiles= predictions_with_irradiance_fc_quantiles * (max_power - min_power) + min_power #Rescale to orignal values
        predictions_with_irradiance_original_quantiles = predictions_with_irradiance_original_quantiles * (max_power - min_power) + min_power #Rescale to orignal values

        
        ### Summ over 2nd dimension (two outputs)
        #if multi_output:
            #predictions_with_irradiance_fc_quantiles = np.sum(predictions_with_irradiance_fc_quantiles, axis=1) #+ (scalings_max_min[1]/(scalings_max_min[0]-scalings_max_min[1]))
            #predictions_with_irradiance_original_quantiles = np.sum(predictions_with_irradiance_original_quantiles, axis=1) #+ (scalings_max_min[1]/(scalings_max_min[0]-scalings_max_min[1]))
        #    predictions_with_irradiance_fc_quantiles = predictions_with_irradiance_fc_quantiles[:, -1, :] #+ (scalings_max_min[1]/(scalings_max_min[0]-scalings_max_min[1]))
        #    predictions_with_irradiance_original_quantiles = predictions_with_irradiance_original_quantiles[:, -1, :] #+ (scalings_max_min[1]/(scalings_max_min[0]-scalings_max_min[1]))
        
        ###
        lower_bound = 0
        upper_bound = -1
        groundtruth = groundtruth_LV
        index = -1
        lower_bound_with_irradiance_fc = predictions_with_irradiance_fc_quantiles[:, index, lower_bound]
        upper_bound_with_irradiance_fc = predictions_with_irradiance_fc_quantiles[:, index, upper_bound]
        lower_bound_with_irradiance_original = predictions_with_irradiance_original_quantiles[:,index, lower_bound]
        upper_bound_with_irradiance_original = predictions_with_irradiance_original_quantiles[:,index, upper_bound]

        uncertainty_interval_irradiance_fc = (upper_bound_with_irradiance_fc - lower_bound_with_irradiance_fc) #/ (1 + np.abs(predictions_with_irradiance_fc))
        uncertainty_interval_irradiance_original = (upper_bound_with_irradiance_original - lower_bound_with_irradiance_original) #/ (1 + np.abs(predictions_with_irradiance_original))

        if show_figures:
            fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

            visualization_idx_start = 0
            visualization_idx_stop = visualization_idx_start + steps2predict

            fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop], 
                                    y= groundtruth[visualization_idx_start:visualization_idx_stop],
                                    name = "Groundtruth"), row=1, col=1)
            fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop], 
                                    y= predictions_with_irradiance_fc_quantiles[visualization_idx_start:visualization_idx_stop,index, 5], 
                                    name = "Prediction"), row=1, col=1)
            # Add lower boundary line
            fig.add_trace(go.Scatter(
                x=time[visualization_idx_start:visualization_idx_stop],
                y=predictions_with_irradiance_fc_quantiles[visualization_idx_start:visualization_idx_stop, index, lower_bound],
                mode='lines',
                line=dict(width=0.5, color='darkblue'),
                showlegend=False,
                name='Lower Bound'
            ), row =1, col=1)

            # Add upper boundary line with fill to the lower boundary line
            fig.add_trace(go.Scatter(
                x=time[visualization_idx_start:visualization_idx_stop],
                y=predictions_with_irradiance_fc_quantiles[visualization_idx_start:visualization_idx_stop, index, upper_bound],
                mode='lines',
                fill='tonexty', # This fills the area between this line and the previous line
                fillcolor='rgba(0, 0, 139, 0.2)', # Adjust the fill color and alpha here
                line=dict(width=0.5, color='darkblue'),
                showlegend=False,
                name='Upper Bound'
            ), row=1, col=1)
            # ad title

            #fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop], y= (upper_bound_with_irradiance_original-lower_bound_with_irradiance_original)[visualization_idx_start:visualization_idx_stop],name = "Uncertainty intervall real weather"))
            fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop], 
                                    y= uncertainty_interval_irradiance_fc[visualization_idx_start:visualization_idx_stop], 
                                    name = "Uncertainty intervall"),
                                    row=2,col=1)
            fig.update_layout(title_text="Prediction with forecasted weather")
            fig.show()

            fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

            visualization_idx_start = 0
            visualization_idx_stop = visualization_idx_start + steps2predict

            fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop], y= groundtruth[visualization_idx_start:visualization_idx_stop],name = "Groundtruth"), row=1, col=1)
            fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop], y= predictions_with_irradiance_original_quantiles[visualization_idx_start:visualization_idx_stop, index, 5], name = "Real weather"), row=1, col=1)
            # Add lower boundary line
            fig.add_trace(go.Scatter(
                x=time[visualization_idx_start:visualization_idx_stop],
                y=predictions_with_irradiance_original_quantiles[visualization_idx_start:visualization_idx_stop, index,  lower_bound],
                mode='lines',
                line=dict(width=0.5, color='darkblue'),
                showlegend=False,
                name='Lower Bound'
            ), row=1, col=1)

            # Add upper boundary line with fill to the lower boundary line
            fig.add_trace(go.Scatter(
                x=time[visualization_idx_start:visualization_idx_stop],
                y=predictions_with_irradiance_original_quantiles[visualization_idx_start:visualization_idx_stop, index, upper_bound],
                mode='lines',
                fill='tonexty', # This fills the area between this line and the previous line
                fillcolor='rgba(0, 0, 139, 0.2)', # Adjust the fill color and alpha here
                line=dict(width=0.5, color='darkblue'),
                showlegend=False,
                name='Upper Bound'
            ), row=1, col=1)

            fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop], 
                                    y= uncertainty_interval_irradiance_original[visualization_idx_start:visualization_idx_stop], 
                                    name = "Uncertainty intervall"),
                                    row=2, col=1)
            fig.update_layout(title_text="Prediction with real weather")
            fig.show()


            fig = go.Figure()

            visualization_idx_start = 0
            visualization_idx_stop = visualization_idx_start + steps2predict


            fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop], y= uncertainty_interval_irradiance_original[visualization_idx_start:visualization_idx_stop],name = "Uncertainty intervall real weather"))
            fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop], y= uncertainty_interval_irradiance_fc[visualization_idx_start:visualization_idx_stop], name = "Uncertainty intervall fc weather"))
            # Add lower boundary line
            # ad title
            fig.update_layout(title_text="Prediction with forecasted weather")
            fig.show()
        
        # for NP, EC, LV:
        # Evaluate squared error, absolute error for forecasted and original irradiance and write to dataframe
        # Evaluate percentage of beeing in uncertainty bounds for forecasted and original irradiance and write to dataframe
        #columns of dataframe: MAE, RMSE, p_in_bound, mean_interval_width, month
        # Two dataframes for forecasted and original irradiance
            
        #predictions_with_irradiance_fc_quantiles # (ts_length, 3, n_quantiles)  #index 5 qunatile is median
        df_rows_month = pd.DataFrame(columns= ['MAE', 'RMSE', 'MEAN_uncertainty_interval', 'PERCENTAGE_in_uncertainty_bounds', 'source', 'month', 'irradiance_type']) 
        for k, source in enumerate(source_order):
            if source == 'NP':
                groundtruth = groundtruth_NP
            elif source == 'EC':
                groundtruth = groundtruth_EC
            elif source == 'LV':
                groundtruth = groundtruth_LV
            
            squared_error_forecast  = (predictions_with_irradiance_fc_quantiles[:,k,5]       - groundtruth)**2
            squared_error_original  = (predictions_with_irradiance_original_quantiles[:,k,5] - groundtruth)**2
            absolute_error_forecast = np.abs(groundtruth - predictions_with_irradiance_fc_quantiles[:,k,5])
            absolute_error_original = np.abs(groundtruth - predictions_with_irradiance_original_quantiles[:,k,5])

            #absolute_percentage_eror_forecast = np.abs((groundtruth - predictions_with_irradiance_fc)/groundtruth)
            #absolute_percentage_eror_original = np.abs((groundtruth - predictions_with_irradiance_original)/groundtruth)

            mse_original_irradiance = squared_error_original.mean()
            mse_forecast_irradiance = squared_error_forecast.mean()

            mae_original_irradiance = absolute_error_original.mean()
            mae_forecast_irradiance = absolute_error_forecast.mean()
            rmse_original_irradiance = math.sqrt(mse_original_irradiance)
            rmse_forecast_irradiance = math.sqrt(mse_forecast_irradiance)

            lower_bound_with_irradiance_fc = predictions_with_irradiance_fc_quantiles[:,k, lower_bound]
            upper_bound_with_irradiance_fc = predictions_with_irradiance_fc_quantiles[:,k, upper_bound]
            lower_bound_with_irradiance_original = predictions_with_irradiance_original_quantiles[:,k, lower_bound]
            upper_bound_with_irradiance_original = predictions_with_irradiance_original_quantiles[:,k, upper_bound]

            uncertainty_interval_irradiance_fc = (upper_bound_with_irradiance_fc - lower_bound_with_irradiance_fc) #/ (1 + np.abs(predictions_with_irradiance_fc))
            uncertainty_interval_irradiance_original = (upper_bound_with_irradiance_original - lower_bound_with_irradiance_original) #/ (1 + np.abs(predictions_with_irradiance_original))

            percentage_in_interval_original_irradiance = (np.logical_and(groundtruth >= lower_bound_with_irradiance_original,  groundtruth <= upper_bound_with_irradiance_original)).mean()
            percentage_in_interval_fc_irradiance = (np.logical_and(groundtruth >= lower_bound_with_irradiance_fc,  groundtruth <= upper_bound_with_irradiance_fc)).mean()
            df_row = pd.DataFrame({'MAE'    : [mae_forecast_irradiance,  mae_original_irradiance] , 
                                'RMSE'   : [rmse_forecast_irradiance, rmse_original_irradiance],
                                'MEAN_uncertainty_interval': [uncertainty_interval_irradiance_fc.mean(), uncertainty_interval_irradiance_original.mean()], 
                                'PERCENTAGE_in_uncertainty_bounds': [percentage_in_interval_fc_irradiance, percentage_in_interval_original_irradiance],
                                'source': [source, source], 'month': [months[i], months[i]], 'irradiance_type': ['forecasted', 'original']})
            df_rows_month = pd.concat([df_rows_month, df_row], ignore_index=True)
        #print(df_rows_month.shape)
        df_metrics_comparison.append(df_rows_month)

    df_metrics_comparison = pd.concat(df_metrics_comparison, ignore_index=True)

else:
    print('no qunatile loss experiment\n Execute next cell')


no qunatile loss experiment
 Execute next cell


In [12]:
if loss_type == 'mse':
    df_metrics_comparison = []
    show_figures = False
    for i in range(6):
        lookback = 96
        test_values_LV = df_test_scaled_list_LV[i].values
        groundtruth_LV = test_values_LV[lookback+1::, 0] # There is a + 1 shift in the inference loop
        groundtruth_NP = df_test_scaled_list_NP[i].values[lookback+1::, 0]
        groundtruth_EC = df_test_scaled_list_EC[i].values[lookback+1::, 0]
    
        num_test_samples = test_values_LV.shape[0]
        steps2predict = num_test_samples-lookback
        
        #groundtruth = test_values[lookback:-1, 0] 
        time = pd.Series(df_test_scaled_list_LV[i].index)[lookback:-1]
        predictions_with_irradiance_fc, predictions_with_irradiance_fc_all = run_inference(model, test_values_LV, lookback=lookback, 
                                                                                                            future_prediction=steps2predict, 
                                                                                                            use_positional_encoding='all', 
                                                                                                            multi_output=multi_output, 
                                                                                                            )
    
        predictions_with_irradiance_original, predictions_with_irradiance_original_all = run_inference(model, test_values_LV, lookback=lookback, 
                                                                                                                        future_prediction=steps2predict,    
                                                                                                                        use_positional_encoding='all_original', 
                                                                                                                        multi_output=multi_output,
                                                                                                                        )
    
        groundtruth_LV = groundtruth_LV * (max_power - min_power) + min_power #Rescale to orignal values
        groundtruth_NP = groundtruth_NP * (max_power - min_power) + min_power #Rescale to orignal values
        groundtruth_EC = groundtruth_EC * (max_power - min_power) + min_power #Rescale to orignal values
    
        predictions_with_irradiance_fc = predictions_with_irradiance_fc * (max_power - min_power) + min_power #Rescale to orignal values
        predictions_with_irradiance_original = predictions_with_irradiance_original * (max_power - min_power) + min_power #Rescale to orignal values
    
        predictions_with_irradiance_fc_all= predictions_with_irradiance_fc_all * (max_power - min_power) + min_power #Rescale to orignal values
        predictions_with_irradiance_original_all = predictions_with_irradiance_original_all * (max_power - min_power) + min_power #Rescale to orignal values
    
    
        groundtruth = groundtruth_LV
        index = -1
    
        if show_figures:
            fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
            visualization_idx_start = 0
            visualization_idx_stop = visualization_idx_start + steps2predict
            fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop], 
                                     y= groundtruth[visualization_idx_start:visualization_idx_stop],
                                     name = "Groundtruth"), row=1, col=1)
            fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop], 
                                     y= predictions_with_irradiance_fc_all[visualization_idx_start:visualization_idx_stop, index], 
                                     name = "Prediction"), row=1, col=1)
            fig.update_layout(title_text="Prediction with forecasted weather")
            fig.show()
           
            fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
            visualization_idx_start = 0
            visualization_idx_stop = visualization_idx_start + steps2predict
            fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop],
                                     y= groundtruth[visualization_idx_start:visualization_idx_stop],
                                     name = "Groundtruth"), row=1, col=1)
            fig.add_trace(go.Scatter(x =time[visualization_idx_start:visualization_idx_stop], 
                                     y= predictions_with_irradiance_original_all[visualization_idx_start:visualization_idx_stop, index], 
                                     name = "Real weather"), row=1, col=1)
            fig.update_layout(title_text="Prediction with real weather")
            fig.show()
    
        # for NP, EC, LV:
        # Evaluate squared error, absolute error for forecasted and original irradiance and write to dataframe
        # Evaluate percentage of beeing in uncertainty bounds for forecasted and original irradiance and write to dataframe
        #columns of dataframe: MAE, RMSE, p_in_bound, mean_interval_width, month
        # Two dataframes for forecasted and original irradiance
            
        #predictions_with_irradiance_fc_quantiles # (ts_length, 3, n_quantiles)  #index 5 qunatile is median
        df_rows_month = pd.DataFrame(columns= ['MAE', 'RMSE', 'source', 'month', 'irradiance_type']) 
        for k, source in enumerate(source_order):
            if source == 'NP':
                groundtruth = groundtruth_NP
            elif source == 'EC':
                groundtruth = groundtruth_EC
            elif source == 'LV':
                groundtruth = groundtruth_LV
            
            squared_error_forecast  = (predictions_with_irradiance_fc_all[:,k]       - groundtruth)**2
            squared_error_original  = (predictions_with_irradiance_original_all[:,k] - groundtruth)**2
            absolute_error_forecast = np.abs(groundtruth - predictions_with_irradiance_fc_all[:,k])
            absolute_error_original = np.abs(groundtruth - predictions_with_irradiance_original_all[:,k])
    
    
            mse_original_irradiance = squared_error_original.mean()
            mse_forecast_irradiance = squared_error_forecast.mean()
    
            mae_original_irradiance = absolute_error_original.mean()
            mae_forecast_irradiance = absolute_error_forecast.mean()
            rmse_original_irradiance = math.sqrt(mse_original_irradiance)
            rmse_forecast_irradiance = math.sqrt(mse_forecast_irradiance)
    
            df_row = pd.DataFrame({'MAE'    : [mae_forecast_irradiance,  mae_original_irradiance] , 
                                   'RMSE'   : [rmse_forecast_irradiance, rmse_original_irradiance],
                                   'source': [source, source], 'month': [months[i], months[i]], 'irradiance_type': ['forecasted', 'original']})
            df_rows_month = pd.concat([df_rows_month, df_row], ignore_index=True)
        #print(df_rows_month.shape)
        df_metrics_comparison.append(df_rows_month)
    
    df_metrics_comparison = pd.concat(df_metrics_comparison, ignore_index=True)
    

In [13]:
df_metrics_comparison.to_csv(os.path.join(lstm_model_path, f'comparison_metrics_{target_model}.csv'))

### Combine all datasets of experiments 

In [11]:
df_train_real_irradiance_quantile = pd.read_csv("./experiments/Rural-LV1-101-2034/BaseScenario/0101-3112/analysis/lstm/model/comparison_metrics_EC_NP_LV_first_real_irradiance.csv")
df_train_forecast_irradiance_quantile = pd.read_csv("./experiments/Rural-LV1-101-2034/BaseScenario/0101-3112/analysis/lstm/model/comparison_metrics_EC_NP_LV_first.csv")

df_train_real_irradiance_mse = pd.read_csv("./experiments/Rural-LV1-101-2034/BaseScenario/0101-3112/analysis/lstm/model/comparison_metrics_EC_NP_LV_first_real_irradiance_MSE.csv")
df_train_forecast_irradiance_mse= pd.read_csv("./experiments/Rural-LV1-101-2034/BaseScenario/0101-3112/analysis/lstm/model/comparison_metrics_EC_NP_LV_first_MSE.csv")

df_train_real_irradiance_mse['train_irradiance'] = 'original'
df_train_forecast_irradiance_mse['train_irradiance'] = 'forecasted'
df_train_real_irradiance_quantile['train_irradiance'] = 'original'
df_train_forecast_irradiance_quantile['train_irradiance'] = 'forecasted'

df_train_real_irradiance_mse['loss_type'] = 'mse'
df_train_forecast_irradiance_mse['loss_type'] = 'mse'
df_train_real_irradiance_quantile['loss_type'] = 'quantile'
df_train_forecast_irradiance_quantile['loss_type'] = 'quantile'

df_combined = pd.concat([df_train_real_irradiance_mse, df_train_forecast_irradiance_mse, df_train_real_irradiance_quantile, df_train_forecast_irradiance_quantile], ignore_index=True)


In [13]:
df_combined.to_csv(os.path.join(lstm_model_path, f'df_summary_combined.csv'))

In [10]:
#NP_test0 = pd.read_csv('./experiments/Rural-LV1-101-2034/BaseScenario/0101-3112/analysis/lstm/data/NP_scaled_by_LV_train.csv')
#EC_test0 = pd.read_csv('./experiments/Rural-LV1-101-2034/BaseScenario/0101-3112/analysis/lstm/data/EC_scaled_by_LV_train.csv')
#LV_test0 = pd.read_csv('./experiments/Rural-LV1-101-2034/BaseScenario/0101-3112/analysis/lstm/data/LV_scaled_by_LV_train.csv')

#NP_test0 = pd.read_csv('./experiments/Rural-LV1-101-2034/BaseScenario/0101-3112/analysis/lstm/data/NP_test0.csv')
#EC_test0 = pd.read_csv('./experiments/Rural-LV1-101-2034/BaseScenario/0101-3112/analysis/lstm/data/EC_test0.csv')
#LV_test0 = pd.read_csv('./experiments/Rural-LV1-101-2034/BaseScenario/0101-3112/analysis/lstm/data/LV_test0.csv')

#df_new = pd.DataFrame( [NP_test0['NP'], EC_test0['EC'], LV_test0['LV']]).T
#x_max =  97.75610212478624 
#x_min = -285.4394146080334
#
##df_new['EC+NP'] = (df_new['NP'] + df_new['EC']) + (x_min / (x_max - x_min))
#
#df_new['LV_orig'] = (df_new['LV']) #*(x_max - x_min) + x_min
#df_new['NP_orig'] = (df_new['NP']) #*(x_max - x_min) + x_min
#df_new['EC_orig'] = (df_new['EC']) #*(x_max - x_min) + x_min
##df_new['EC+NP_orig'] = (df_new['NP_orig'] + df_new['EC_orig'])
##df_new['LV/NP'] = (df_new['LV'] / df_new['NP']) #+ (x_min / (x_max - x_min))
##df_new['LV/EC'] = (df_new['LV'] / df_new['EC']) #+ (x_min / (x_max - x_min))
#px.line(df_new).show()


In [17]:

df_train_real_irradiance = pd.read_csv("./experiments/Rural-LV1-101-2034/BaseScenario/0101-3112/analysis/lstm/model/comparison_metrics_EC_NP_LV_real_irradiance_MSE.csv")
df_train_forecast_irradiance = pd.read_csv("./experiments/Rural-LV1-101-2034/BaseScenario/0101-3112/analysis/lstm/model/comparison_metrics_EC_NP_LV_forecasted_irradiance_MSE.csv")

df_train_real_irradiance_InterpolLin = pd.read_csv("./experiments/Rural-LV1-101-2034/BaseScenario_FCInterpolLin/0101-3112/analysis/lstm/model/comparison_metrics_EC_NP_LV_real_irradiance_MSE.csv")
df_train_forecast_irradiance_InterpolLin = pd.read_csv("./experiments/Rural-LV1-101-2034/BaseScenario_FCInterpolLin/0101-3112/analysis/lstm/model/comparison_metrics_EC_NP_LV_forecasted_irradiance_MSE.csv")

df_train_real_irradiance['train_irradiance'] = 'original'
df_train_forecast_irradiance['train_irradiance'] = 'forecasted'
df_train_real_irradiance_InterpolLin['train_irradiance'] = 'original_InterpolLin'
df_train_forecast_irradiance_InterpolLin['train_irradiance'] = 'forecasted_InterpolLin'

#df_train_real_irradiance['loss_type'] = 'mse'
#df_train_forecast_irradiance['loss_type'] = 'mse'

df_combined = pd.concat([df_train_real_irradiance, df_train_forecast_irradiance, df_train_real_irradiance_InterpolLin, df_train_forecast_irradiance_InterpolLin], ignore_index=True)


In [23]:
df_combined_LV = df_combined[df_combined['source'] == 'LV']
df_combined_LV

,Unnamed: 0,MAE,RMSE,source,month,irradiance_type,train_irradiance
4,4,14.974258,26.108735,LV,july,forecasted,original
5,5,7.858742,12.082875,LV,july,original,original
10,10,21.540101,39.647330,LV,august,forecasted,original
11,11,9.347223,13.982174,LV,august,original,original
16,16,18.712162,30.745548,LV,september,forecasted,original
17,17,11.259826,17.256246,LV,september,original,original
22,22,12.057777,19.090721,LV,october,forecasted,original
23,23,9.797462,14.577116,LV,october,original,original
28,28,10.946331,16.310599,LV,november,forecasted,original
29,29,10.507123,13.484029,LV,november,original,original


In [27]:
fig = px.line(df_combined_LV, x='month', y='RMSE', color='train_irradiance', title='RMSE', symbol = 'irradiance_type')
fig.show()
fig.write_html('./export_plot_MSE_comparison.html')